# Chapter 5

# K-Nearest Neighbors Method

## 5.1 A Basic k-NN Classifier

### 5.1.1 The _"Can I eat that?" App_

We start by exploring the mushrooms dataset and looking at a random sample of the data and understand what is there

In [1]:
import pandas as pd

data = pd.read_csv("../datasets/mushrooms.csv")
data.sample(10)

,E,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21
7219,p,k,s,e,f,f,f,c,n,b,...,s,w,p,p,w,o,e,w,v,l
3931,p,f,f,y,f,f,f,c,b,g,...,k,p,p,p,w,o,l,h,v,g
6597,p,f,s,n,f,s,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
4767,p,f,f,g,f,f,f,c,b,g,...,k,p,b,p,w,o,l,h,y,p
343,e,f,f,y,t,a,f,w,n,p,...,s,w,w,p,w,o,p,n,v,d
4175,p,f,f,g,f,f,f,c,b,g,...,k,p,p,p,w,o,l,h,y,d
6135,p,k,s,e,f,f,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
4338,p,x,y,g,f,f,f,c,b,g,...,k,n,b,p,w,o,l,h,v,d
7518,p,k,y,n,f,s,f,c,n,b,...,s,p,w,p,w,o,e,w,v,l
479,e,b,y,y,t,l,f,c,b,n,...,s,w,w,p,w,o,p,k,s,m


---
After inspecting the data, we find that we're only intereted in the features labeled as `"F0", "F1", "F2", "F20", "F21"`, so we extract those as our data of interest along with, of course, the label `"E"`

In [2]:
data_of_interest = data.loc[:, ["E", "F0", "F1", "F2", "F20", "F21"]]

---
### 5.1.3 How to Measure Similarity?
Before we can measure the Euclidean distance between two mushrooms, we need to map the charcter codes for the features to numeric values in order to plug them into the equation 

In [3]:
def numerically_encode(df):
    
    encoded_df = df.copy()
    encoders = {}
    
    for col in df.columns:
        unique_categories = df.loc[:, col].unique()
        unique_categories.sort() # in-place sorting
    
        encoder = {str: num for num, str in enumerate(unique_categories)}
        encoders[col] = encoder
    
        encoded_df.loc[:, col] = df.loc[:, col].apply(lambda x: encoder[x])
    
    return encoded_df, encoders

---
We now encode our data and reterive the encoder dictionary for the label column to see how each label is encoded and be able to interpret the results of the classifier we're going to build.

In [4]:
encoded_data, encoders = numerically_encode(data_of_interest)
print(encoders["E"])

{'e': 0, 'p': 1}


---
### 5.1.4 k-NN in Action
We start our implementation of the k-NN model by implementing a function that calculates the Euclidean distance between two mushrooms 

In [5]:
import math

def d(p1, p2):
    
    N = len(p1)
    squared_distance = 0
    
    for i in range(N):
        squared_distance += (p1[i] - p2[i]) ** 2
        
    return math.sqrt(squared_distance)

---
We can now implement the k-NN classifier by simply calculating the distances between the input we have and all the data in the training set, sort those in ascending fashion by the distance, pick the first k and then report the most common label in our chosen set of k data points.

In [6]:
from collections import Counter

def knn_classifier(new_x, k, X_train, y_train):

    neighbors = [] # a list of tuples (distance_to_new_x, neighbor_label)
    for x, y in zip(X_train, y_train):
        distance = d(x, new_x)
        neighbors.append((distance, y))

    sorted_neighbors = sorted(neighbors, key=lambda n: n[0])
    nearest_k_neighbors = sorted_neighbors[:k]
    
    labels_counter = Counter([label for _, label in nearest_k_neighbors])
    most_voted_label = max(labels_counter.items(), key=lambda i: i[1])
    
    return most_voted_label[0]

---
Before we can test the k-NN classifier, which we did before in chapters 1, 3 with scikit-learn. However, as long as we're doing things from scratch, let's see how we can do it manually

In [7]:
shuffled_data = encoded_data.sample(frac=1., random_state=42)
X, y = shuffled_data.loc[:, 'F0':], shuffled_data.loc[:, "E"]
X, y = X.as_matrix(), y.as_matrix()

X_train, y_train = X[:6125], y[:6125]
X_test, y_test = X[6125:], y[6125:]

---
We can now run our classifier and see it in action and how it performs on the test set

In [8]:
test_preds = [knn_classifier(x, 5, X_train, y_train) for x in X_test]

losses = [1. if y_pred != y else 0. for y_pred, y in zip(test_preds, y_test)]
test_error = (1. / len(test_preds)) * sum(losses)
accuracy = 1. - test_error

print("Test Error {:.3f}, Test Accuracy: {:.3f}".format(test_error, accuracy))

Test Error 0.110, Test Accuracy: 0.890


---
While this implementation is giving us a decent performance, it's ineffcient when it comes to time. We can verfiy that by running the `%timeit` magic against the prediction step and see

In [9]:
%timeit [knn_classifier(x, 5, X_train, y_train) for x in X_test]

1min 20s ± 4.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
### 5.1.5 Boosting Performance with NumPy

NumPy fixes the problem with python that causes the above code's ineffciency. Instead of usind dynamically-typed and scattered containers, NumPy relies on data structures that have static pre-defined types which allow them to occupy contiguous memeory locations. These data startuctures are the `ndarray`, which we can find that they are the backbone of our `pandas` data frames.

In [10]:
print(type(X))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


To start using `ndarray`s and utilize their effciency, we first need to understand one of its charactristics that play a key role in determining its behavior, which is its `shape`

In [11]:
print(X.shape)
print(y.shape)

(8124, 5)
(8124,)


---
Shapes determine if we can perform arithmetic operations on `ndarray`s. If the shapes are _compatible_, then arithmetic operation can be performed, and they are performed element-wise like we see in the following example:

In [12]:
import numpy as np

four_ones = np.ones(shape=(4, ))
powers_of_two = np.array([1, 2, 4, 8])

print(four_ones / powers_of_two)

[ 1.     0.5    0.25   0.125]


---
We can take advantage of the broadcasting feature and rewrite our k-NN implementation bye getting rid of the nested for loops for calculating the distance and transfer them from python to NumPy optimized C/C++ code. We also change the voting code to use NumPy alternative and fully harness its powers

In [13]:
def faster_knn_classifier(new_x, k, X_train, y_train):
    
    neighbors_distances = np.sqrt(np.sum((X_train - new_x) ** 2, axis=1))
        
    sorted_neighbors_indecies = np.argsort(neighbors_distances)
    nearest_k_neighbors_indecies = sorted_neighbors_indecies[:k]
    nearest_k_neighbors_labels = y_train[nearest_k_neighbors_indecies]
    
    labels, votes = np.unique(nearest_k_neighbors_labels, return_counts=True)
    most_voted_label_index = np.argmax(votes)
    
    return labels[most_voted_label_index]

---
Now we can test our new k-NN classifier and see how much faster it has become

In [14]:
faster_test_preds = [
    faster_knn_classifier(x, 1, X_train, y_train) for x in X_test
]
faster_losses = (faster_test_preds != y_test)
faster_test_error = np.mean(faster_losses)

print("Test Error {:.4f}, Test Accuracy: {:.4f}".format(
    faster_test_error, 
    1. - faster_test_error
))

Test Error 0.1176, Test Accuracy: 0.8824


In [15]:
%timeit [faster_knn_classifier(x, 5, X_train, y_train) for x in X_test]

1.41 s ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
## 5.2 A Better k-NN Classifier
### 5.2.2 Using k-d Tress with scikit-learn
After we saw how k-d trees can boost the nearst neighbors search, let's see that in action by using scikit-learn's implementation of the k-NN model that comes with a k-d trees implementation

In [21]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
classifier.fit(X_train, y_train)

sklearn_test_accuracy = classifier.score(X_test, y_test)

print("Test Error {:.4f}, Test Accuracy: {:.4f}".format(
    1. - sklearn_test_accuracy, 
    sklearn_test_accuracy
))

Test Error 0.1176, Test Accuracy: 0.8824


In [22]:
%timeit classifier.score(X_test, y_test)

21.1 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 5.2.3 Tuning the Value of k

To start searching through the possible values of k in order to find the value that would return the best performing model, we need to craete another held out dataset that is used during this serach, thus leaving our test unseen. 

In [23]:
new_X_train, X_valid = X_train[:5525], X_train[5525:]
new_y_train, y_valid = y_train[:5525], y_train[5525:]

---
With this new **_validation set_**, we can start looking through the best k value by iterating through the possible values and training a k-NN classifier and recording the best performing one on the training set

In [24]:
best_score, best_k, best_classifier = 0., None, None
for k in range(1, 21):
        classifier = KNeighborsClassifier(n_neighbors=k)
        classifier.fit(new_X_train, new_y_train)
        score = classifier.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_k = k
            best_classifier = classifier

print("Best k: {}, Best Validation Score: {:.4f}".format(best_k, best_score))
print("Test Accuracy: {:.4f}".format(best_classifier.score(X_test, y_test)))

Best k: 11, Best Validation Score: 0.8950
Test Accuracy: 0.9005


---
We can extend our linear search tuning algorithm into a grid search algorithm that searches through pairs of hyperparameters and reports the best configuration. The second hyperparameter we're going to search for is the `metric` and we're gonna see that automatic choice aligns with our expert's judgment

In [25]:
best_score, best_k, best_metric, best_classifier = 0., -1, None, None
for k in range(1, 21):
    for metric in ['euclidean', 'manhattan', 'hamming', 'canberra']:
        classifier = KNeighborsClassifier(n_neighbors=k, metric=metric)
        classifier.fit(new_X_train, new_y_train)
        score = classifier.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_k = k
            best_metric = metric
            best_classifier = classifier

print("Best k: {}, Best Metric: {}, Best Validation Score: {:.4f}".format(
    best_k, 
    best_metric, 
    best_score
))
print("Test Accuracy: {:.4f}".format(best_classifier.score(X_test, y_test)))

Best k: 11, Best Metric: hamming, Best Validation Score: 0.9017
Test Accuracy: 0.9065
